In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
from secret import *
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [2]:
graph = Neo4jGraph(
    url = "bolt://localhost:7687",
    username = "neo4j",
    password = "health1234"
)

In [3]:
graph.query(
"""
MATCH (n)
DETACH DELETE n
"""
)

[]

In [4]:
import pandas as pd
df = pd.read_csv('health-kg.csv')

In [6]:
import re
def get_sp_dict(symptoms):
    symptoms = symptoms.split(", ")
    s_p_dict = {}
    for symptom in symptoms:
        s, p = symptom.split(" (")
        p = float(
            re.sub(
                "[^0-9A-Za-z.\s\:']",
                "",
                p,
            )
        )
        s_p_dict[s] = p
    return s_p_dict

In [7]:
def camel_case(s):
    # Use regular expression substitution to replace underscores and hyphens with spaces,
    # then title case the string (capitalize the first letter of each word), and remove spaces
    s = re.sub(r"(_|-)+", " ", s).title().replace(" ", "")
    
    # Join the string, ensuring the first letter is lowercase
    return ''.join([s[0].lower(), s[1:]])

In [8]:
from tqdm import tqdm

In [9]:
query = "CREATE "

In [26]:
df["Diseases"]

0                          abscess
1                      acid reflux
2              acute renal failure
3             alcohol intoxication
4                       alcoholism
                  ...             
151                type 2 diabetes
152             ulcerative colitis
153    upper respiratory infection
154        urinary tract infection
155           vitamin d deficiency
Name: Diseases, Length: 156, dtype: object

In [10]:
for i, d in tqdm(enumerate(df["Diseases"]), total=len(df)):
    d = re.sub(
        "[^0-9A-Za-z ]",
        "",
        d,
    )

    query += "(%s:Disease {name: '%s'})," % (
        camel_case(d),
        d,
    )

    # if i+1 < len(df):
    #     query +=","
        


100%|██████████| 156/156 [00:00<?, ?it/s]


In [11]:
# graph.query(query)

In [12]:
all_symptoms = []
for sp_dict in list(df["Symptoms"].apply(get_sp_dict)):
    all_symptoms.extend(list(sp_dict.keys()))

In [13]:
all_symptoms =set(all_symptoms)

In [14]:
len(all_symptoms)

330

In [15]:
for i, s in tqdm(enumerate(all_symptoms), total=len(all_symptoms)):
    s = re.sub(
        "[^0-9A-Za-z ]",
        "",
        s,
    )

    query += "(%s:Symptom {name: '%s'})," % (
        camel_case(s),
        s,
    )

    # if i+1 < len(all_symptoms):
    #     query +=","

100%|██████████| 330/330 [00:00<00:00, 329317.23it/s]


In [16]:
# graph.query(query)

In [17]:
for i, data in tqdm(df.iterrows(), total=len(df)):
    d = re.sub(
        "[^0-9A-Za-z ]",
        "",
        data["Diseases"],
    )

    symptoms = data["Symptoms"]
    sp_dict = get_sp_dict(symptoms)
    for s, p in sp_dict.items():
        query += "(%s)-[:HAS {prob: %f}]->(%s)," % (
            camel_case(d),
            p,
            camel_case(s),
        )

100%|██████████| 156/156 [00:00<00:00, 9793.03it/s]


In [18]:
graph.query(query[:-1])  # remove last comma

[]

In [ ]:
# MATCH p=()-[r:HAS]->(fever:Symptom {name: "fever"}) RETURN p LIMIT 25
# MATCH p=(abscess:Disease {name:"abscess"})-[r:HAS]->() RETURN p LIMIT 25